# `1.7 RF 📡`

## <span style="color:tomato"> Overview </span>

The SX1272 is a transceiver integrated circuit (chip) that takes digital messages from the microcontroller and is capable of modulating them to a particular communication protocol such that they can be transmited and received as data packets with the antenna on the LunaSat. 

---

## <span style="color:tomato"> RF's Role In the Mission </span>

RF is necessary not only to send data to the lander, but also to make sure that all LunaSats can maintain accurate timestamps for qualifying data via clock synchronization algorithms. You will discover that the RF system is the most power-intensive component on our LunaSat later in this notebook. To address this, we anticipate switching between two "modes" on the LunaSat on the Moon - one where data collection is occuring and sensors are active, and another where only necessary components and the RF components are active. Essentially, RF will never be active as long as sensors are active, and vice versa.

#### <span style="color:orange">Example Configuration Timeline</span>

#### | <span style="color:orange">**Temperature + Accelerometer On, RF Off**</span> (10 seconds) | <span style="color:orange">**RF On, All Sensors Off**</span> (5 seconds) | <span style="color:orange">**Temperature On, RF Off**</span> (7 seconds) |

In reality, each LunaSat will be assigned a transmit / receive window during which RF is on and sensors are off. This means that your final choice of configurations will not actually include any RF configurations. Additionally, RF parameters will be optimized on our end so as to ensure the success of network operations.

This notebook will ultimately show you GLEE's process of determining how to configure the RF system so that its power usage is minimized. It will also explore ensuring that RF will work based on the landing location of the LunaSats.

---

## <span style="color:tomato"> Configurations </span>

#### The SX1272 chip is designed to work with a particular communications protocol called LoRa™. There are several LoRa™ operating modes to choose from. Don't worry if you don't understand the jargon used below. We will highlight which modes are important later. 
- <span style="color:orange">**SLEEP**</span>: A low-power mode where only SPI and configuration registers are active and accessible.
    - Typically uses 0.1 µA and up to a max of 1 µA.
- <span style="color:orange">**STANDBY**</span>: The crystal oscillator and LoRa™ baseband blocks are turned on, while the RF front-end and PLLs are disabled.
    - Typically uses 1.4 mA and up to 1.6 mA
- <span style="color:orange">**IDLE**</span>
    - Typically uses 1.5 µA.
- <span style="color:orange">**FSTX**</span>: A frequency-synthesis mode for transmission, where the PLL selected for transmission is locked and active at the transmit frequency while the RF front-end is off
    - Typically uses 4.5 mA.
- <span style="color:orange">**FSRX**</span>: A frequency-synthesis mode for receiving, where the PLL selected for reception is locked and active at the receive frequency while the RF front-end is off
    - Typically uses 4.5 mA.
- <span style="color:orange">**TX**</span>: All remaining blocks required for transmitting are powered, the PA is ramped and the packet is sent. The SX1272 then goes into STANDBY mode.
    - Typically uses 90 mA at +17dBm with an external impedance transformation. 
- <span style="color:orange">**RXCONTINUOUS**</span>: All remaining blocks required for receiving are powered and data that is received is processed. Done continuously until the mode is changed.
    - Typically uses 11.6 mA with LnaBoost on and with a bandwidth of 250 kHz.
- <span style="color:orange">**RXSINGLE**</span>: All remaining blocks required for receiving are powered and data that is received is processed. Once a packet is received, the SX1272 goes into STANDBY mode.
    - Typically uses 11.6 mA with LnaBoost on and with a bandwidth of 250 kHz.
- <span style="color:orange">**CAD**</span>: The device will check a given channel to detect a LoRa preamble signal.

The modes we are particularly interested in are <span style="color:orange">SLEEP</span>, <span style="color:orange">STANDBY</span>, <span style="color:orange">TX</span>, <span style="color:orange">RXCONTINUOUS</span>, and <span style="color:orange">RXSINGLE</span>. <span style="color:orange">SLEEP</span> will be the primary mode for when data collection is occuring and we want the RF system to effectively be off. <span style="color:orange">STANDBY</span> will help to reduce power usage when transmissions are not actively occuring. <span style="color:orange">RXSINGLE</span> may also help to reduce power usage in cases where the LunaSat is not expecting data spread across multiple packets.

---

## <span style="color:tomato"> Creating Configurations </span>

- <span style="color:orange">**Modes**</span>: {"SLEEP", "STANDBY", "TX", "RXCONTINUOUS", "RXSINGLE", "IDLE", "FSTX", "FSRX", "CAD"}

#### There are a number of LoRa™ parameters we can also configure the SX1272 to have. You may recognize that these parameters were utilized in Module 9.

- <span style="color:orange">**Frequency**</span>: {Any value between 860 and 1000 (1 GHz)}
    - Frequency of radio waves received or transmitted by RF Module. Typically 915-928MHz corresponding to LoRas operation on unlicensed ISM Radio Band in US. Set to 915 MHz in our code.
- <span style="color:orange">**Output Power**</span>: { Any value between 0 and 20}
    - Power of transceiver transmission. Set to 17 dBm in our code. The higher the output power, the farther a signal will be able to travel before becoming undetectable by a receiver.
- <span style="color:orange">**Bandwidth**</span>: {125,250,500}
    - Bandwidth is the frequency range of chirps used to transmit data. Typically 250kHz for terrestrial usage in the US. Set to 250 kHz in our code.
- <span style="color:orange">**Spreading Factor**</span>: {7, 8, 9, 10, 11, 12}
    - The Spreading Factor dictates the number of chirps used to send a signal. Set to 12 in our code.
- <span style="color:orange">**Coding Rate**</span>: {5,6,7,8}
    - Coding Rate effects data encoding redundancies. CR value is proportional to interference tolerance. Set to 8 in our code. Denoted as 4/#. (Higher - Able to handle more interference)

Certain configurations do not utilize all available settings. Below are the configuration formats for the 3 possible modes you can choose from. Anything with {} braces indicates a parameter you can change.

 - <span style="color:orange">("SLEEP", 0, 0, 0, 0, 0)</span>
 - <span style="color:orange">("STANDBY", { Frequency }, { Output Power }, { Bandwidth }, { Spreading Factor }, { Coding Rate })</span>
 - <span style="color:orange">("IDLE", { Frequency }, { Output Power }, { Bandwidth }, { Spreading Factor }, { Coding Rate })</span>
 - <span style="color:orange">("FSTX", { Frequency }, { Output Power }, { Bandwidth }, { Spreading Factor }, { Coding Rate })</span>
 - <span style="color:orange">("FSRX", { Frequency }, { Output Power }, { Bandwidth }, { Spreading Factor }, { Coding Rate })</span>
 - <span style="color:orange">("TX", { Frequency }, { Output Power }, { Bandwidth }, { Spreading Factor }, { Coding Rate })</span>
 - <span style="color:orange">("RXCONTINUOUS", { Frequency }, { Output Power }, { Bandwidth }, { Spreading Factor }, { Coding Rate })</span>
 - <span style="color:orange">("RXSINGLE", { Frequency }, { Output Power }, { Bandwidth }, { Spreading Factor }, { Coding Rate })</span>
 - <span style="color:orange">("CAD", { Frequency }, { Output Power }, { Bandwidth }, { Spreading Factor }, { Coding Rate })</span>

You should also choose how long the sensor will spend in that configuration (duration, in seconds) and how fast the TMP117 sensor should retrieve samples (in seconds between samples).
- <span style="color:orange">**Configuration duration (s)**</span>: {n} 
- <span style="color:orange">**Estimated Transmission Rate (s)**</span>: {n}
    - The time between transmissions. Will only be used for modes when transmission is active. Realistically, this will be variable and dependent on many factors, including the amount of data that needs to be transmitted. The necessary time to transmit a packet is TODO. So the estimated transmission rate must be no smaller than TODO.
- <span style="color:orange">**Estimated Reception Rate (s)**</span>: {n}
    - The time between receiving packets. Will only be used for modes when reception is active. Realistically, this will be variable and dependent on many factors, including if packets are expected at all and how much data is expected to be received. The necessary time to receive a packet is TODO. So the estimated reception rate must be no smaller than TODO.

## <span style="color:tomato"> Configurations For A Lunar Mission </span>

---

## <span style="color:tomato"> Calculating Power Usage </span>

## <span style="color:tomato"> Putting It All Together </span>

[Click here to continue to the next notebook.](2.0%20Combined%20PDM.ipynb)